# DARPA AMP Numerical Methods Challenge
This challenge scenario is to patch a theoretically correct routine that suffers from convergence issues based on the input. The algorithm is the Newton-Raphson method and the problem it's solving is orbit trajectory calculations based on Kepler's laws.

Note: This challenge is inspired from site visits with NASA Goddard.

## Scenario
A satellite or space probe may have a highly elliptial orbit. These orbits are calculated using Newton's and Kepler's laws. The use of Kepler's equation is shown in https://en.wikipedia.org/wiki/Kepler%27s_equation and many other references regarding orbital mechanics. It is used onboard a space craft to determine its location in orbit. 

Kepler's equation is:

$$ M = E - e \sin E $$

where $M$ is the mean anomaly, $E$ is the eccentric anomaly, and $e$ is the eccentricity.

Kepler's equation is a transcendental equation, which means there is not an algebraic solution for E. Instead, we must use a numerical method, like the Newton-Raphson method. To do this, we are looking for a root, so let's transform the equation to 
$$ f(E) = E - e \sin(E) - M$$ and

$$ f^\prime(E) = 1 - e \cos(E) $$

Let's start with an initial guess $E_0$, then use the Newton-Raphson method to determine a solution. When $f = 0$, the solution is satisfied. The numerical method should converge so $f<tol$.

$$E_n = E_{n-1} - \frac{f(E_{n-1})}{f^\prime(E_{n-1})}$$
Typically this convergence is fast and only takes 3-6 iterations. However, if the tolerance is too tight, and we are using discrete representation of real numbers (i.e. floating point math), then sometimes the convergence criterion can be missed.

### Example
Let's look an an example of solving Kepler's equation using Python. We'll run 2 examples: one that converges quickly, and one that doesn't satisfy the (overly precise) convergence criteria.

The convergence criteria for this is the machine's floating point epsilon.

The the difference in the input is 0.000001

The input is 

 * mean_anomaly = 6.2
 * eccentricity = .335999 or .3360
 
We'll use the mean_anomaly as our initial guess for E.


In [2]:
from math import sin, cos
import sys

In [2]:
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [3]:
sys.float_info.epsilon

2.220446049250313e-16

In [4]:
#Newton–Raphson formulations for the Kepler equation
# M = E - e sin E
#where M is the mean anomaly, E is the eccentric anomaly, and e is the eccentricity. 
def formula(ecc_anomaly,eccentricity,mean_anomaly):
    return ecc_anomaly - eccentricity*(sin(ecc_anomaly)) - mean_anomaly

def formula_derivative(ecc_anomaly,eccentricity):
    return 1- eccentricity*(cos(ecc_anomaly))

In [5]:
itter = 50
tol = sys.float_info.epsilon
mean_anomaly = 6.2 #radians
for eccentricity in [.335999, .3360]:
    print("Input: e = {}, M = {}".format(eccentricity,mean_anomaly))
    error = 1
    E = [mean_anomaly]
    n = 0
    while abs(error) > tol:
        error = formula(E[-1],eccentricity,mean_anomaly)
        E.append(E[-1] - error/formula_derivative(E[-1],eccentricity))
        print(error)
        #Add this to prevent infinite loops.
        n+=1
        if n >= itter:
            print("Did not converge.")
            break
    print("The solution converged to M = {} radians after {} iterations with error of {}.\n".format(E[-1],n,error))

Input: e = 0.335999, M = 6.2
0.027917956257275556
-2.8712578192369165e-05
-3.8900438426026085e-11
0.0
The solution converged to M = 6.158071460674133 radians after 4 iterations with error of 0.0.

Input: e = 0.336, M = 6.2
0.02791803934667847
-2.8712939017516703e-05
-3.8901326604445785e-11
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197001252e-16
-8.881784197001252e-16
8.881784197

# Perceptor Plan
We need to convert this concept into a challenge for DARPA AMP performers. To do this we have to come up with the following:
1. Vulnerable source code
2. Patched source code
3. Vulnerable binary
4. Patched binary

There are many ways we can generate this quad of deliverables based on some vulnerable code, hardware, and tool chains.

## Patch Definition
The vulnerable code and resulting binary have issues in calculating the convergence criteria for some inputs. Then this happens, the algorithm could go into an infinite loop. There are a few ways of overcome this:
1. Add an iteration counter and exit the calculation loop if a specified number of iterations has been exceeded.
2. Add a timeout and exit the calculation loop if a certain amount of time has passed. 
3. Relax the precision of the error tolerance. 
4. Restart with alternative initial conditions. 

### Iteration Counter
To implement the first check, the condition could be built into the while statement, or the condition could be checked during the iteration and exit if stasfied. For example, either of the following snippets would satisfy the patch 1.

In [6]:
itter = 4
n = 0
error = 1
while error > tol and n < itter:
    n+=1
    print(n)

1
2
3
4


In [7]:
itter = 4
n = 0
error = 1
while error > tol :
    n+=1
    print(n)
    if n >= itter:
        break

1
2
3
4


General application of this method may require a timing analysis of the algorithm, which could change based on hardware, clock speed, and the initial conditions. 

### Relax the Precision
Instead of using machine epsilon, we could use an alternative criteria. For example, 10 or 100 times machine epsilon. 

In [10]:
tol = 10*sys.float_info.epsilon
itter = 10
mean_anomaly = 6.2 #radians
for eccentricity in [.335999, .3360]:
    print("Input: e = {}, M = {}".format(eccentricity,mean_anomaly))
    error = 1
    E = [mean_anomaly]
    n = 0
    while abs(error) > tol:
        error = formula(E[-1],eccentricity,mean_anomaly)
        E.append(E[-1] - error/formula_derivative(E[-1],eccentricity))
        
        #Add this to prevent infinite loops.
        n+=1
        if n >= itter:
            print("Did not converge.")
            break
    print("The solution converged to M = {} radians after {} iterations with error of {}.\n".format(E[-1],n,error))

Input: e = 0.335999, M = 6.2
The solution converged to M = 6.158071460674133 radians after 4 iterations with error of 0.0.

Input: e = 0.336, M = 6.2
The solution converged to M = 6.15807127348128 radians after 4 iterations with error of -8.881784197001252e-16.



### Change initial conditions
This approach is difficult by itself because the algorithm would need to know when to change. To determine if a change was necessary, we would likely employ one of the approaches above. 

# Challenge Problem Suite
DARPA performers will have 3 hardware platforms this fall with a PowerPC platform being built. The CAN Logger 3 is also available. 

We need to build the following things:
1. An input/output system. The theme has been CAN, so we can probably stick with that.
  1. When using CAN, we'll need to define some messages. I suspect eliptical orbits are not defined in J1939, so we can use a proprietary PGN.
  1. We'll need a CAN message to start the calculation. Let's use the CAN message with the ID of 0x18FF81F9 and the data fields are 2 32-bit, Big Endian numbers that represent the eccentricity and the mean anomaly. 
  1. The reponse should be the value of the response to Kepler's Equation. Let's use 0x18FF821C to respond. This uses the source address of 28 (0x1C) for vehicle navigation. We can use a 32-bit unsigned integer in big endian format for the response. We could also include a byte on the number of iterations it took, and finally an indicator of the value of the error in the computation. 
2. A function that performs the Newton-Raphson calculation. This is the function that has the vulnerability.
3. The the formula that is used to solve Kepler's equation.
4. The formula for the derivative of Kepler's equation.


## Hardware

1. Arduino Uno: This is a great option since the floating point has to be done in 8-bit code.
1. STM32F04: Basic ARM-32 processor, no floating point processors.
1. CAN Logger 3 / NXP K66: ARM 32-bit processor with floating point unit.
4. BeagleBone Black: Embedded Linux on ARM
5. MPC57XX development board: PowerPC e200 core. 
5. PowerPC Emulator: Will we be able to compile and run code for the the emulator?
6. CM2350: Cummins engine controller, PowerPC. The N-R method would have to be added by Cummins and flashed to the Controller. Maybe this is a stretch goal, but it would be pretty cool.
7. NASA inspired hardware... 

What architectures and emulation environments are available for us to implement challenge problems on?

## Build Chains
1. Arduino IDE for the Arduino Uno
1. STMCube for the STM32
1. Arduino with Teensyduino (gcc) for the CAN Logger 3. We could also use the NXP tools.
4. Linux GCC and SocketCAN for the beagle bone.
  1. Could also write code with rust, go, c, and c++
5. The MPC57XX would use the NXP tools.
6. Assuming Cummins will use their tool chain? Can we run custom code?
7. CM2350 emulator: ???
8. NASA inspired tool chains... 